# 8x8 Matrix: Accuracy, RT, Task, Duration 

Input data is split randomly in half ~10,000 times

In [36]:
import os 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy 
import scipy.stats as stats
from scipy.stats import sem 
from sklearn.linear_model import LinearRegression
import pickle
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import random

In [37]:
final_discrim_path = '/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/depth_discrimination/TAC_discrim_datafiles/matched_discrim_data/final_discrim.csv'


In [38]:
final_discrim = pd.read_csv(final_discrim_path)


In [39]:
n_VE_data_250_path = '/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/verbal_judgement_analysis/TAC_data/matched_VE_data/normalized_250_data.csv'

n_VE_data_1000_path = '/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/verbal_judgement_analysis/TAC_data/matched_VE_data/normalized_1000_data.csv'


In [40]:
n_VE_data_250 = pd.read_csv(n_VE_data_250_path)
n_VE_data_1000 = pd.read_csv(n_VE_data_1000_path)


In [187]:
def get_stimuli(duration_df):
    """
    Args:
        duration df 
    Returns:
        all_stimuli
    """
    
    stimuli = set()
    for idx, row in duration_df.iterrows():
        stimulus = row["stimulus"]
        stimuli.add(stimulus)
        
    return list(stimuli)

def rand_data(duration_df, proportion):
    """
    Args
        Data post outlier removal: i.e. cleaned_data 
        proportion - amount of data in each split (0.5 for 50/50 split)
    """
    
    stimuli = get_stimuli(duration_df)
    
    rand_y = []
    rand_std = []
    rand_ste = []
    rand_stim = []
    
    rand_all_ys = []
    
    rand_subjIDs = []

    grouped = duration_df.groupby(duration_df.stimulus) 
    for stim in stimuli:   
        stim_df = grouped.get_group(stim)
        stim_df_estimates = stim_df['depth_estimate'].tolist()
        stim_df_subjIDs = stim_df['subjID'].tolist()
        
        # Shuffle two lists with same order
        # Using zip() + * operator + shuffle()
        temp = list(zip(stim_df_subjIDs, stim_df_estimates))
        random.shuffle(temp)
        shuffled_subjIDs, shuffled_estimates = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        shuffled_subjIDs, shuffled_estimates = list(shuffled_subjIDs), list(shuffled_estimates)
        
        rand_all_ys.append(np.array(shuffled_estimates[0:int(len(shuffled_estimates)*proportion)]))
        rand_subjIDs.append(np.array(shuffled_subjIDs[0:int(len(shuffled_subjIDs)*proportion)]))
        
        estim_avg_df0 = np.mean(np.array(shuffled_estimates[0:int(len(shuffled_estimates)*proportion)]))
        estim_std = np.std(np.array(shuffled_estimates[0:int(len(shuffled_estimates)*proportion)]))
        estim_ste = stats.sem(np.array(shuffled_estimates[0:int(len(shuffled_estimates)*proportion)]))
        
        rand_y.append(estim_avg_df0) 
        rand_std.append(estim_std)
        rand_ste.append(estim_ste)
        rand_stim.append(stim)
    
    rand_y = np.array(rand_y).reshape(1,-1)[0]
    rand_std = np.array(rand_std).reshape(1,-1)[0]
    rand_ste = np.array(rand_ste).reshape(1,-1)[0]
    rand_stim = np.array(rand_stim).reshape(1,-1)[0]
    
    rand_df = pd.DataFrame(columns=('subjID', 'stimulus', 'depth_estimate'))
    
    c = 0
    # loop through all stimuli
    for i, stim in enumerate(rand_stim):
        # loop through all the subjects that have data for that stimulus
        for j in range(len(rand_subjIDs[i])):
            # add to the df the subject id, the subjects estimate, and the stimulus name
            # df has a seperate row for each subjects response 
            rand_df.loc[c] = [rand_subjIDs[i][j], stim, rand_all_ys[i][j]]
            c += 1
            
    return [rand_y, rand_std, rand_ste, rand_stim], rand_df



In [188]:
r0_250, r0_250_df = rand_data(n_VE_data_250, 0.5)
r0_1000, r0_1000_df = rand_data(n_VE_data_1000, 0.5)

In [289]:
n_VE_data_250

,Unnamed: 0,subjID,experimentName,versionName,sequenceName,url,selected_row,windowWidth,windowHeight,screenWidth,...,duration,actual_depth,depth_estimate,trial_RT,log_sceneDuration,unitSelection,experimentTime,totalTime,age,gender
0,0,810087,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_6.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,111,1440,757,1440,...,250,3.9835,1.431018,4443,250,feet,1682664,1792976,40,Woman
1,1,810087,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_6.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,111,1440,757,1440,...,250,1.5580,1.788773,5052,250,feet,1682664,1792976,40,Woman
2,2,810087,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_6.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,111,1440,757,1440,...,250,3.5340,1.073264,4196,249,feet,1682664,1792976,40,Woman
3,3,810087,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_6.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,111,1440,757,1440,...,250,1.6565,1.252141,3365,256,feet,1682664,1792976,40,Woman
4,4,810087,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_6.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,111,1440,757,1440,...,250,4.0370,0.894386,3746,248,feet,1682664,1792976,40,Woman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9648,23994,754962,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_8.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,127,1536,722,1536,...,250,4.0585,1.513295,6098,250,feet,1177038,1334470,60,Woman
9649,23995,754962,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_8.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,127,1536,722,1536,...,250,2.8490,1.297110,5041,243,feet,1177038,1334470,60,Woman
9650,23996,754962,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_8.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,127,1536,722,1536,...,250,2.1070,0.972832,5996,250,feet,1177038,1334470,60,Woman
9651,23997,754962,DepthScenes,duration_manipulation_targetAtFixation,jsons/VE250_randls_8.json,http://54.173.230.142/FacialAge/BNav_EC2/Depth...,127,1536,722,1536,...,250,2.3540,1.080925,5232,251,feet,1177038,1334470,60,Woman


In [48]:
def get_accuracy(df):
    """
    * ACCURACY BASED ON THE KINECT
    args:
        df  
    returns:
        proportion of correct responses, count of correct responses, count of total trials  
    """
    count_correct = 0
    count_incorrect = 0
    count_total = 0
    count_missed = 0
    for idx, row in df.iterrows():
        choice = row["discrim_choice"]
        if choice == 2.0:
            count_missed += 1
        else:    
            count_total += 1
            depth0 = row["actual_depth_0"]
            depth1 = row["actual_depth_1"]
            if depth0 < depth1:
                correct_choice = 0
            if depth0 > depth1:
                correct_choice = 1
            if depth0 == depth1:
                # case where depths are equal 
                correct_choice = None
            if choice == correct_choice:
                count_correct += 1
    
    return count_correct/count_total, count_correct, count_total, count_missed

def get_RT(df):
    """
    Converts raw RT --> ln(RT)
    
    args:
        df  
    returns:
        array of RTs, avg RT and std   
    """
    list_RTs = []
    for idx, row in df.iterrows():   
        stimulus_duration = row['log_sceneDuration2']
        RT = row["trial_RT"] - stimulus_duration
        list_RTs.append(np.log(RT))
    
    list_RTs = np.array(list_RTs)
    
    return list_RTs, np.mean(list_RTs) ,np.std(list_RTs), stats.sem(list_RTs)

In [49]:
def individual_discrimination_stats(df):
    '''
    Individual discrimination performance and RT 
    '''
    all_stim0 = df.stimulus_0.unique()
    
    stimuli_stats = {}
    for stim0 in all_stim0:
        stim0_df = df.loc[df['stimulus_0'] == stim0]
        other_stim = stim0_df.stimulus_1.unique()[0]
        stim1_df = df.loc[df['stimulus_1'] == stim0]
        # df for a specific discrimination trial (collapsed on stim presentation order)
        stim_df = pd.concat([stim0_df, stim1_df], ignore_index=True)
        stim_250_df = stim_df[stim_df['duration'] == 250.0]
        stim_1000_df = stim_df[stim_df['duration'] == 1000.0] 
                
        stim_depthdiff = stim_df['depth_difference'][0]
        
        stim0_depth = stim_df['actual_depth_0'][0]
        stim1_depth = stim_df['actual_depth_1'][0]
        stim_depthbin = np.mean(np.array([stim0_depth,stim1_depth]))
        
        kinect_answer = stim0_df.kinect_answer.unique()[0]
        
        try:
            stim_acc_250 = get_accuracy(stim_250_df)
            stim_acc_1000 = get_accuracy(stim_1000_df)

            stim_RT_250 = get_RT(stim_250_df)
            stim_RT_1000 = get_RT(stim_1000_df)

            stimuli_stats[stim0] = {'stimulus_1': other_stim,
                                    'accuracy_250': stim_acc_250,
                                    'accuracy_1000': stim_acc_1000,
                                    'avg_depth': stim_depthbin,
                                    'depthdifference': stim_depthdiff, 
                                    'RT_250': stim_RT_250,
                                    'RT_1000': stim_RT_1000,
                                    'kinect_answer': kinect_answer}
        except:
            pass
    
    return stimuli_stats

In [76]:
all_discrim_performance = individual_discrimination_stats(final_discrim)


In [53]:
# all_discrim_performance['depth_discrimination_stimuli/001417_2014-06-19_16-25-36_260595134347_rgbf000115-resize_5/001417_2014-06-19_16-25-36_260595134347_rgbf000115-resize_5-target.png']


In [65]:
def get_answerkey(discrim_performance, VE_data):
    '''
    Generates discrimination trial answer key based off of verbal judgement data
    '''
    answerkey = {}
    
    final_y = VE_data[0]
    final_std = VE_data[1]
    final_ste = VE_data[2]
    final_stim = VE_data[3]

    for key in discrim_performance.keys():
        targetimg0 = key.split('/')[-1]
        folder0 = targetimg0[:-11]
        depth_dur_path0 = 'depth_duration_stimuli/' + folder0 + '/' + targetimg0
        idx0 = np.where(final_stim == depth_dur_path0)[0][0]
        avg_estim_stim0 = final_y[idx0]
        std0 = final_std[idx0]
        ste0 = final_ste[idx0]
    
        targetimg1 = discrim_performance[key]['stimulus_1'].split('/')[-1]
        folder1 = targetimg1[:-11]
        depth_dur_path1 = 'depth_duration_stimuli/' + folder1 + '/' + targetimg1
        idx1= np.where(final_stim == depth_dur_path1)[0][0]
        avg_estim_stim1 = final_y[idx1]
        std1 = final_std[idx1]
        ste1 = final_ste[idx1]
    
        kinect_answer = discrim_performance[key]['kinect_answer'].split('/')[-1]

        if avg_estim_stim0 < avg_estim_stim1:
            # Which target is CLOSER to you?
            answer = targetimg0
        if avg_estim_stim0 == avg_estim_stim1:
            print(targetimg0, targetimg1)
        if avg_estim_stim0 > avg_estim_stim1:
            answer = targetimg1

        answerkey[key] = {'stimulus_1': targetimg1,
                                   'stimulus_0_avg_estim': avg_estim_stim0,
                                   'stimulus_1_avg_estim': avg_estim_stim1,
                                   'answer': answer,
                                   'std0': std0,
                                   'std1': std1,
                                   'kinect_answer': kinect_answer}
        
    return answerkey
    

In [77]:
ak_250 = get_answerkey(all_discrim_performance, r0_250)
ak_1000 = get_answerkey(all_discrim_performance, r0_1000)

### VE Coded Accuracy

In [85]:
def VE_accuracy(stim0, df, answerkey):
    '''
    Accuracy based on the verbal judgement data 
    '''
    
    """
    args:
        df  
    returns:
        proportion of correct responses, count of correct responses, count of total trials  
    """
    count_correct = 0
    count_incorrect = 0
    count_total = 0
    count_missed = 0
    
    VE_correct_answer = answerkey[stim0]['answer']
    kinect_correct_answer = answerkey[stim0]['kinect_answer']
    
    for idx, row in df.iterrows(): 
        choice = row["discrim_choice"]
        count_total += 1
        if choice == 0.0: image_choice = row["stimulus_0"]
                
        if choice == 1.0: image_choice = row["stimulus_1"]
                
        if choice == 2.0: count_missed += 1
            
        if choice == 3.0: count_missed += 1
        
        try:
            if image_choice.split('/')[-1] == VE_correct_answer: count_correct += 1
        except: pass
        
#     standardError = (0.5*(1-0.5))/count_total
    p = count_correct/count_total
    standardError = np.sqrt((p*(1-p))/count_total)
    
    if VE_correct_answer == kinect_correct_answer:
        return count_correct/count_total, count_correct, count_total, count_missed, standardError, 'pos'
    else:
        return count_correct/count_total, count_correct, count_total, count_missed, standardError, 'neg'

    
def main_VE_coded_discrim_accuracy(df, answerkey_250, answerkey_1000):
    '''
    Args:
        df: final discrimination dataframe
        answerkey_250 & answerkey_1000: VE coded answer key
    '''
    all_stim0 = df.stimulus_0.unique()
    
    stimuli_stats = {}
    for stim0 in all_stim0:
        try:
            # dataframe for stimulus 0
            stim0_df = df.loc[df['stimulus_0'] == stim0]
            # name of stimulus 1
            other_stim = stim0_df.stimulus_1.unique()[0]
            # dataframe where stimulus 0 is presented SECOND (same trial)
            stim1_df = df.loc[df['stimulus_1'] == stim0]

            # df for a specific discrimination trial (collapsed on stim presentation order)
            stim_df = pd.concat([stim0_df, stim1_df], ignore_index=True)
            stim_250_df = stim_df[stim_df['duration'] == 250.0]
            stim_1000_df = stim_df[stim_df['duration'] == 1000.0] 

            stim0_depth = stim_df['actual_depth_0'][0]
            stim1_depth = stim_df['actual_depth_1'][0]
            stim_depthbin = np.mean(np.array([stim0_depth,stim1_depth]))

            stim_acc_250 = VE_accuracy(stim0, stim_250_df, answerkey_250)
            stim_acc_1000 = VE_accuracy(stim0, stim_1000_df, answerkey_1000)

            stim_RT_250 = get_RT(stim_250_df)
            stim_RT_1000 = get_RT(stim_1000_df)

            # difference between verbal judgements divided by joint variance 
            # abs(VE1-VE2)/sqrt(stda^2 + std2^2)
            std0_250 = answerkey_250[stim0]['std0']
            std1_250 = answerkey_250[stim0]['std1']
            joint_variance_250 = np.sqrt(std0_250**2 + std1_250**2)
            JV_regressor_250 = abs(answerkey_250[stim0]['stimulus_0_avg_estim'] - answerkey_250[stim0]['stimulus_1_avg_estim'])/joint_variance_250

            std0_1000 = answerkey_1000[stim0]['std0']
            std1_1000 = answerkey_1000[stim0]['std1']
            joint_variance_1000 = np.sqrt(std0_1000**2 + std1_1000**2)
            JV_regressor_1000 = abs(answerkey_1000[stim0]['stimulus_0_avg_estim'] - answerkey_1000[stim0]['stimulus_1_avg_estim'])/joint_variance_1000
            
            if stim_acc_250[-1] == 'pos':
                VE_depthdifference_250 = abs(answerkey_250[stim0]['stimulus_0_avg_estim'] - answerkey_250[stim0]['stimulus_1_avg_estim'])
            else:
                VE_depthdifference_250 = -(abs(answerkey_250[stim0]['stimulus_0_avg_estim'] - answerkey_250[stim0]['stimulus_1_avg_estim']))
            
            if stim_acc_1000[-1] == 'pos':
                VE_depthdifference_1000 = abs(answerkey_1000[stim0]['stimulus_0_avg_estim'] - answerkey_1000[stim0]['stimulus_1_avg_estim'])
            else:
                VE_depthdifference_1000 = -(abs(answerkey_1000[stim0]['stimulus_0_avg_estim'] - answerkey_1000[stim0]['stimulus_1_avg_estim']))
            
            stimuli_stats[stim0] = {'stimulus_1': other_stim,
                                    'accuracy_250': stim_acc_250,
                                    'accuracy_1000': stim_acc_1000,
                                    'avg_depth': stim_depthbin,
                                    'VE_depthdifference_250': VE_depthdifference_250, 
                                    'VE_depthdifference_1000': VE_depthdifference_1000,
                                    'RT_250': stim_RT_250,
                                    'RT_1000': stim_RT_1000,
                                    'JV_regressor_250': JV_regressor_250,
                                    'JV_regressor_1000': JV_regressor_1000}
    
        except:
            print(stim0)

    return stimuli_stats
    

In [88]:
VE_coded_discrim_performance = main_VE_coded_discrim_accuracy(final_discrim, ak_250, ak_1000)

In [90]:
def get_discrimination_results(discrim_performance, duration):
    '''
    Returns lists of discrimination data results
    '''
    if duration == 250:
        n_VE_estim_diff = [discrim_performance[elem]['VE_depthdifference_250'] for elem in discrim_performance]
        n_VE_accuracy = [discrim_performance[elem]['accuracy_250'][0] for elem in discrim_performance]
        n_VE_ste = [discrim_performance[elem]['accuracy_250'][-2] for elem in discrim_performance]
        n_avg_RT = [discrim_performance[elem]['RT_250'][1] for elem in discrim_performance]
        n_avg_RT_ste = [discrim_performance[elem]['RT_250'][-1] for elem in discrim_performance]
        n_JV = [discrim_performance[elem]['JV_regressor_250'] for elem in discrim_performance]
        
    if duration == 1000:
        n_VE_estim_diff = [discrim_performance[elem]['VE_depthdifference_1000'] for elem in discrim_performance]
        n_VE_accuracy = [discrim_performance[elem]['accuracy_1000'][0] for elem in discrim_performance]
        n_VE_ste = [discrim_performance[elem]['accuracy_1000'][-2] for elem in discrim_performance]
        n_avg_RT = [discrim_performance[elem]['RT_1000'][1] for elem in discrim_performance]
        n_avg_RT_ste = [discrim_performance[elem]['RT_1000'][-1] for elem in discrim_performance]
        n_JV = [discrim_performance[elem]['JV_regressor_1000'] for elem in discrim_performance]

    n_stim = [elem for elem in discrim_performance]      
    
    return n_VE_estim_diff, n_VE_accuracy, n_VE_ste, n_avg_RT, n_avg_RT_ste, n_JV, n_stim

In [91]:
discrimination_results_250 = get_discrimination_results(VE_coded_discrim_performance, 250)
discrimination_results_1000 = get_discrimination_results(VE_coded_discrim_performance, 1000)

In [94]:
len(discrimination_results_250)

7

### Derive Performance for VE 

In [274]:
def VE_performance(stimuli, duration_df, discrimination_performance, duration_answerkey):
    '''
    Args:
        stimuli:  depth discrimination stimuli list 
        duration_df: VE duration dataframe with ONLY stimulus and list of depth estimates
        discrimination performance: dictionary of discrim performance 
    '''
    
    VE_Performance = {}
            
    for im0 in stimuli:
        performance = []
        # loop through all participants
        for subjID in duration_df.subjID.unique():
            # filter to just the subjects data df
            subjdf = duration_df.loc[duration_df['subjID'] == subjID]
            
            try:
                im0_VE = 'depth_duration_stimuli/' + im0.split('/')[1] + '/' + im0.split('/')[2]
                im0_row = subjdf.loc[subjdf['stimulus'] == im0_VE]
                im0_estimate = im0_row['depth_estimate'].tolist()[0]

                im1 = discrimination_performance[im0]['stimulus_1'][29:]
                im1_VE = 'depth_duration_stimuli/' + im1
                im1_row = subjdf.loc[subjdf['stimulus'] == im1_VE]
                im1_estimate = im1_row['depth_estimate'].tolist()[0]
                

                if im0_estimate < im1_estimate:
                    p_ans = im0.split('/')[-1]
                else:
                    p_ans = im1.split('/')[-1]
                try:
                    answerkey_answer = duration_answerkey[im0]['answer']
                except:
                    answerkey_answer = duration_answerkey['depth_discrimination_stimuli/' + im1]['answer']
                if p_ans == answerkey_answer:
                    trial_acc = 0 # CORRECT
                    performance.append(trial_acc)
                else:
                    trial_acc = 1 # INCORRECT
                    performance.append(trial_acc)
            
            except:
                pass
        VE_Performance[im0] = performance

    VE_PC = {}

    for key in VE_Performance:
        performance = VE_Performance[key]
        correct_count = performance.count(0)
        incorrect_count = performance.count(1)
        total = len(performance)
        pc = correct_count/total
        VE_PC[key] = pc
    
    return VE_PC
    
    

In [275]:
VE_performance_250 = VE_performance(discrimination_results_250[-1], r0_250_df, VE_coded_discrim_performance, ak_250)
VE_performance_1000 = VE_performance(discrimination_results_1000[-1], r0_1000_df, VE_coded_discrim_performance, ak_1000)

### Matched Discrimination Performance

In [281]:
def matched_performance(VE_Performance, discrimination_performance, duration):
    '''
    Args:
        VE_performance: dict of VE performance for a specific duration
        discrimination_performance: complete discrim performance
        duration: int, duration value
    Returns:
        Trial matched VE and Discrimination performance
        List of stimuli 
        
    '''
    Discrim_VEmatched_PC = {}
    all_stim = []
    all_VE_PC = []
    all_Discrim_VEmatched_PC = []

    for key in VE_Performance:
        
        im_VE_PC = VE_Performance[key]
        if duration == 250:
            im_Discrim_PC = discrimination_performance[key]['accuracy_250'][0]
        if duration == 1000:
            im_Discrim_PC = discrimination_performance[key]['accuracy_1000'][0]
        Discrim_VEmatched_PC[key] = [im_VE_PC, im_Discrim_PC]

        all_stim.append(key)
        all_VE_PC.append(im_VE_PC)
        all_Discrim_VEmatched_PC.append(im_Discrim_PC)
        
    return all_Discrim_VEmatched_PC, all_VE_PC, all_stim

In [286]:
m_discrim_performance_250, m_VE_performance_250, m_stim_250 = matched_performance(VE_performance_250, VE_coded_discrim_performance, 250)
m_discrim_performance_1000, m_VE_performance_1000, m_stim_1000 = matched_performance(VE_performance_1000, VE_coded_discrim_performance, 1000)

In [287]:
m_discrim_performance_250

[0.5348837209302325,
 0.6585365853658537,
 0.9523809523809523,
 0.717948717948718,
 0.8837209302325582,
 0.8809523809523809,
 0.95,
 0.8,
 0.7857142857142857,
 0.375,
 0.925,
 0.1794871794871795,
 0.4,
 0.9767441860465116,
 0.9024390243902439,
 0.925,
 0.9069767441860465,
 0.7142857142857143,
 0.825,
 0.8,
 0.7317073170731707,
 0.975,
 0.725,
 0.9,
 0.9534883720930233,
 0.6190476190476191,
 0.5116279069767442,
 0.6190476190476191,
 0.5238095238095238,
 0.9285714285714286,
 0.9047619047619048,
 0.6666666666666666,
 0.813953488372093,
 0.8837209302325582,
 0.5853658536585366,
 0.5476190476190477,
 0.9047619047619048,
 0.525,
 0.6046511627906976,
 0.625,
 0.8604651162790697,
 0.7368421052631579,
 0.6410256410256411,
 0.5609756097560976,
 0.5384615384615384,
 0.7142857142857143,
 0.7380952380952381,
 0.9069767441860465,
 1.0,
 0.8048780487804879,
 0.9285714285714286,
 0.4634146341463415,
 0.9302325581395349,
 0.46511627906976744,
 0.6585365853658537,
 0.225,
 0.926829268292683,
 0.85714285

### Plot